In [1]:
import ipywidgets as widgets
from IPython.display import clear_output, display, Image
import os
!git clone https://github.com/xinyu1205/recognize-anything.git
%cd recognize-anything
!git checkout ec6b4241c5036e337a4543838deb9bff4990de97
!pip install timm transformers fairscale pycocoevalcap

Cloning into 'recognize-anything'...
remote: Enumerating objects: 737, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 737 (delta 310), reused 290 (delta 224), pack-reused 290 (from 1)
Receiving objects: 100% (737/737), 27.14 MiB | 23.63 MiB/s, done.
Resolving deltas: 100% (397/397), done.
/content/recognize-anything
Note: switching to 'ec6b4241c5036e337a4543838deb9bff4990de97'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at ec6b424 update


In [2]:
model = "Tag2Text"

In [3]:
def download_checkpoints(model):
    print('You selected', model)
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    if model == "RAM":
        ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
        if not os.path.exists(ram_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
        else:
            print("RAM weights already downloaded!")
    else:
        tag2text_weights_path = 'pretrained/tag2text_swin_14m.pth'
        if not os.path.exists(tag2text_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/tag2text_swin_14m.pth -O pretrained/tag2text_swin_14m.pth
        else:
            print("Tag2Text weights already downloaded!")

download_checkpoints(model)
print(model, 'weights are downloaded!')


You selected Tag2Text
--2024-09-11 11:44:03--  https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/tag2text_swin_14m.pth
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.17, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/xinyu1205/recognize-anything/resolve/main/tag2text_swin_14m.pth [following]
--2024-09-11 11:44:03--  https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/tag2text_swin_14m.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e6/78/e678f8565485a3f321b1180e4c7e1e18a89a9295028358eedffb98981b37e11a/4ce96f0ce98f940a6680d567f66a38ccc9ca8c4e638e5f5c5c2e881a0e3502ac?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tag2text_swin_14m.pth%3B+filename%3D%22tag2text_swin_1

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
images_dir = "/content/drive/MyDrive/DAM"

In [11]:
image_files = [f"{images_dir}/{file}" for file in sorted(os.listdir(images_dir)) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]
image_path = image_files[0]

# Create dropdown widget
image_dropdown = widgets.Dropdown(
    options=image_files,
    description='Select Image:',
)

# Create image preview widget
image_preview = widgets.Output()

# Define function to update image preview
def update_preview(change):
    global image_path
    image_path = change.new
    with image_preview:
        image_preview.clear_output()
        display(Image(filename=image_path, width=400))

# Set the initial image preview
with image_preview:
    display(Image(filename=image_files[0], width=400))

# Attach the update function to the dropdown
image_dropdown.observe(update_preview, names='value')

# Display the widgets
display(image_dropdown, image_preview)

Dropdown(description='Select Image:', options=('/content/drive/MyDrive/DAM/download (1).jpg', '/content/drive/…

Output()

In [16]:
task = "one image" #or can be multiple image
def run_inference(model, task):
    if model == "Tag2Text" and task == "one image":
        !python inference_tag2text.py  --image "{image_path}" \
        --pretrained pretrained/tag2text_swin_14m.pth
    elif model == "Tag2Text" and task == "multiple images":
        !python batch_inference.py --image-dir {images_dir} \
        --pretrained pretrained/tag2text_swin_14m.pth --model-type tag2text
    elif model == "RAM" and task == "one image":
        !python inference_ram.py  --image {image_path} \
        --pretrained pretrained/ram_swin_large_14m.pth
    elif model == "RAM" and task == "multiple images":
        !python batch_inference.py --image-dir {images_dir} \
        --pretrained pretrained/ram_swin_large_14m.pth --model-type ram
    else:
        print('Invalid model or task')

run_inference(model, task)

tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 270kB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 3.10MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 43.9MB/s]
config.json: 100% 570/570 [00:00<00:00, 3.47MB/s]
/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------

In [18]:
task = "multiple images"  # Set to "multiple images" for batch processing
def run_inference(model, task):
    if model == "Tag2Text" and task == "one image":
        # Inference on a single image
        !python inference_tag2text.py  --image "{image_path}" \
        --pretrained pretrained/tag2text_swin_14m.pth
    elif model == "Tag2Text" and task == "multiple images":
        # Batch inference for multiple images
        !python batch_inference.py --image-dir "{images_dir}" \
        --pretrained pretrained/tag2text_swin_14m.pth --model-type tag2text
    elif model == "RAM" and task == "one image":
        # Inference on a single image
        !python inference_ram.py  --image "{image_path}" \
        --pretrained pretrained/ram_swin_large_14m.pth
    elif model == "RAM" and task == "multiple images":
        # Batch inference for multiple images
        !python batch_inference.py --image-dir "{images_dir}" \
        --pretrained pretrained/ram_swin_large_14m.pth --model-type ram
    else:
        print('Invalid model or task')

# Run batch inference
run_inference(model, task)


/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
pretrained/tag2text_swin_14m.pth
--------------
load checkpoint from pretrained/tag2text_swin_14m.pth
vit: swin_b
{'filepath': '/content/drive/MyDrive/DAM/download.jpg', 'model_identified_tags': 'dice | game | white